In [19]:
import scanpy as sc
import numpy as np
import warnings 
warnings.filterwarnings ("ignore")
import re, random
from pathlib import Path
import json
import pandas as pd
from pandas import DataFrame
import os, sys
import torch
import TOSICA
from icecream import ic
ic.configureOutput(includeContext=True, argToStringFunction=lambda _: str(_))
from utils.log_util import logger

root_data_dir = Path('data')
data_type = 'hPancreas'
data_dir = root_data_dir / data_type

Information of GPU

In [20]:
print(torch.cuda.get_device_capability(device=None))
print(torch.cuda.get_device_name(device=None))

(7, 5)
Tesla T4


Reference data

In [21]:
train_file = data_dir / 'train.h5ad'
ref_adata = sc.read(train_file)
gene_names = set(ref_adata.var_names)
logger.info(len(gene_names))
logger.info(ref_adata.var_names[:5])
logger.info(ref_adata.obs_names[:5])
# view of the data
ref_adata = ref_adata[:, ref_adata.var_names]
logger.info(ref_adata)
logger.info(ref_adata.obs.Celltype.value_counts())
logger.info(len(ref_adata.obs.Celltype.value_counts()))
a = ref_adata[['human1_lib1.final_cell_0001', 'human1_lib1.final_cell_0003'], ['COL1A1', 'COL1A2', 'PPY', 'CTRB1']]
logger.info(a)
b = a.X.toarray().tolist()
logger.info(b)

23-05-20 06:51 3980563912.py 4: 3000
23-05-20 06:51 3980563912.py 5: Index(['COL1A1', 'COL1A2', 'PPY', 'CTRB1', 'CELA3A'], dtype='object')
23-05-20 06:51 3980563912.py 6: Index(['human1_lib1.final_cell_0001', 'human1_lib1.final_cell_0003',
       'human1_lib1.final_cell_0004', 'human1_lib1.final_cell_0005',
       'human1_lib1.final_cell_0006'],
      dtype='object')
23-05-20 06:51 3980563912.py 9: View of AnnData object with n_obs × n_vars = 10600 × 3000
    obs: 'Celltype'
    var: 'Gene Symbol'
23-05-20 06:51 3980563912.py 10: Celltype
alpha          3136
beta           2966
ductal         1290
acinar         1144
delta           793
PSC             524
PP              356
endothelial     273
macrophage       52
mast             25
epsilon          21
schwann          13
t_cell            7
Name: count, dtype: int64
23-05-20 06:51 3980563912.py 11: 13
23-05-20 06:51 3980563912.py 13: View of AnnData object with n_obs × n_vars = 2 × 4
    obs: 'Celltype'
    var: 'Gene Symbol'
23-05-

Query data

In [22]:
query_adata = sc.read(data_dir / 'test.h5ad')
logger.info('%s', query_adata.var_names)
query_adata = query_adata[:, query_adata.var_names]
logger.info(query_adata)
logger.info('query_adata.obs.Celltype\n%s', query_adata.obs.Celltype)
logger.info('query_adata.var\n%s', query_adata.var)
logger.info(query_adata.obs.Celltype.value_counts())
logger.info(len(query_adata.obs.Celltype.value_counts()))
logger.info(query_adata.shape)
# type(query_adata.var['Gene Symbol'])) is Series

23-05-20 06:51 1818168239.py 2: Index(['COL1A1', 'COL1A2', 'PPY', 'CTRB1', 'CELA3A', 'CTRB2', 'SPINK1',
       'COL3A1', 'CPB1', 'COL6A3',
       ...
       'DUSP19', 'ZNF160', 'NPRL2', 'CILP2', 'SNED1', 'FAM110D', 'SELL',
       'C9orf135', 'GRIN2D', 'HERC5'],
      dtype='object', length=3000)
23-05-20 06:51 1818168239.py 4: View of AnnData object with n_obs × n_vars = 4218 × 3000
    obs: 'Celltype'
    var: 'Gene Symbol'
23-05-20 06:51 1818168239.py 5: query_adata.obs.Celltype
10th_C11_S96    beta
10th_C13_S61    beta
10th_C14_S53    beta
10th_C17_S97    beta
10th_C19_S62     PSC
                ... 
Sample_1594       PP
Sample_1595       PP
Sample_1597       PP
Sample_1598       PP
Sample_1600       PP
Name: Celltype, Length: 4218, dtype: category
Categories (11, object): ['MHC class II', 'PP', 'PSC', 'acinar', ..., 'ductal', 'endothelial', 'epsilon', 'mast']
23-05-20 06:51 1818168239.py 6: query_adata.var
         Gene Symbol
COL1A1        COL1A1
COL1A2        COL1A2
PPY         

Training:

Prediction

In [23]:
from app.app_utils import read_train_config

data_type = 'hPancreas'
project_dir = Path(f'projects/hGOBP_demo')

model_weight_paths = read_train_config(data_type)
model_weight_path = model_weight_paths[0]
new_adata = TOSICA.pre(query_adata, model_weight_path=model_weight_path, project_path=project_dir)

23-05-20 06:51 app_utils.py 12: data_type hPancreas
23-05-20 06:51 pre.py 46: cuda:0
23-05-20 06:51 pre.py 88: all_line num 4218
23-05-20 06:51 pre.py 97: start_line_i num 0


the X data in predicted new_adata is the attention weight, thats 4218 * 299, the 299 pathways has most genes.

In [24]:
new_adata.write('demo_attn.h5ad')
new_adata

... storing 'Prediction' as categorical
... storing 'Celltype' as categorical
23-05-20 06:51 attrs.py 203: Creating converter from 5 to 3


AnnData object with n_obs × n_vars = 4218 × 299
    obs: 'Prediction', 'Probability', 'Celltype'

Training on pre-weights of 'human_gobp' mask:

In [31]:
celltype = ("alpha","beta","ductal","acinar","delta","PP","PSC","endothelial","epsilon","mast","macrophage","schwann",'t_cell')
print(sorted(new_adata.obs['Prediction'].unique()))
print(sorted(celltype))
print(len(new_adata.obs['Prediction'].unique()))
print(len(celltype))

print(len(new_adata.obs['Celltype'].unique()))
print('Real labels, Celltype')
print(sorted(new_adata.obs['Celltype'].unique()))
print(new_adata.obs['Celltype'].value_counts())
print(new_adata.obs['Prediction'].value_counts())

['PP', 'PSC', 'acinar', 'alpha', 'beta', 'delta', 'ductal', 'endothelial', 'epsilon', 'macrophage', 'mast', 'schwann']
['PP', 'PSC', 'acinar', 'alpha', 'beta', 'delta', 'ductal', 'endothelial', 'epsilon', 'macrophage', 'mast', 'schwann', 't_cell']
12
13
11
Real labels, Celltype
['MHC class II', 'PP', 'PSC', 'acinar', 'alpha', 'beta', 'delta', 'ductal', 'endothelial', 'epsilon', 'mast']
Celltype
alpha           2011
beta            1006
ductal           414
PP               282
acinar           209
delta            188
PSC               73
endothelial       16
epsilon            7
mast               7
MHC class II       5
Name: count, dtype: int64
Prediction
alpha          2012
beta            999
ductal          494
PP              284
delta           182
acinar          133
PSC              70
endothelial      22
epsilon           9
mast              7
macrophage        5
schwann           1
Name: count, dtype: int64


In [26]:
# TOSICA.train(ref_adata, gmt_path='human_gobp', label_name='Celltype',pre_weights='pre_weights.pth',epochs=1)